#Pre-Process GLADE+ Galaxy Catalog

The [GLADE+ Catalog](https://glade.elte.hu/) contains information on ~22.5 million galaxies and ~750,000 quasars. To use this notebook, you will first need to download the catalog as a text file. Here the text file is saved as `GLADE+.txt`.

This code reduces the 40-column spreadsheet to just two columns:
>`Pix`: The pixel location of the celestial object represented by a healpy map with an nside value of 2048. This location is derived from the `RA` (Right Ascension) and `Dec` (Declination) variables in the original dataset.

>`Z_bin`: The "bucket" to which the celestial body is assigned based on the redshift bins created using the [Tomographer interactive tool](http://tomographer.org/guest/jobs#). These bins help categorize galaxies based on their redshift values.

###Imports
>Healpy: for converting `RA` and `Dec` from angle coordinates to pixel coordinates.

>Numpy: for converting between degrees and radians in the pixel coordinate calculation.

>Pandas: for extracting the `z_bin` intervals.

>Tqdm: for tracking progress of the file reading.

In [ ]:
import healpy as hp
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from tqdm import tqdm

###Get Redshift Bin Cutoffs

In [ ]:
z_bins = pd.read_csv('z_bin_cutoffs.csv')
z_bins = z_bins['z']

###Determine the Correct Redshift Bin for Objects

In [ ]:
def get_z_bin(z_val):
    for i in range(len(z_bins)):
        if z_val < z_bins[i]:
            return i

###Set `nside` Parameter for Pixel Coordinate Calculation

`nside` is the resolution parameter determining the number of pixels in the dust map. Higher Nside values result in finer resolution but require more computational resources. Since we use `nside=2048` to generate the dust intensity models, it is important we use the same value here to place the galaxies at the correct relative positions.

In [ ]:
nside = 2048

###Contruct `.csv` File
The following code extracts the Right Ascension: `RA` (column 8) and the Declination: `Dec` (column 9) variables from the original text file so it can later use these values to calculate the pixel location of the object. It also extracts the redshift (column 27) which is used to determine the redshift bin.

This code takes a *long* time to run. The progress bar only tracks the progress of the for-loop; the pixel location calculations take significantly longer.

In [ ]:
ra = []
dec = []
z_bin = []

with open('GLADE+.txt') as f:
    txt = f.readlines()

for line in tqdm(txt):
    line = line.strip()
    e = line.split()
    if e[27] != 'null' and e[8] != 'null' and e[9] != 'null':
        ra.append(float(e[8]))
        dec.append(float(e[9]))
        z_bin.append(get_z_bin(float(e[27])))

coords = SkyCoord(ra=ra,dec=dec,unit='deg',frame='icrs')
pixels = hp.ang2pix(nside=2048,theta=np.pi/2.0 - np.radians(coords.galactic.b.value),phi=np.radians(coords.galactic.l.value),lonlat=False)

data_dict = {
    'Pix': pixels,
    'Z_Bin': z_bin,
}

df = pd.DataFrame(data_dict)
df.to_csv('GLADE+_2048.csv', index=False)